---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [5]:
#! cat readonly/train.csv > train.csv
#! chmod 664 train.csv

In [6]:
#! cat readonly/test.csv > test.csv
#! chmod 664 test.csv

In [9]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import GridSearchCV

def blight_model():

    # read data
    test = pd.read_csv("./test.csv")
    train = pd.read_csv("./train.csv", engine="python")
    
    
    # change zip_code to numeric as I might need it for the prediction 
    train["zip_code"] = pd.to_numeric(train["zip_code"], errors = "coerce")
    test["zip_code"] = pd.to_numeric(test["zip_code"], errors = "coerce")
    
    train.set_index("compliance", inplace=True)
    
    # select the columns present only in the test data set too
    needed_columns=test.columns
    train_X=train.loc[:,needed_columns]
    
    # drop NaN in index and compliance target ID
    train_X = train_X.drop([np.nan], axis='rows').copy()
    
    train_X["city"] = train_X["city"].str.lower() 
    # select only for detroit values
    train_X.loc[(train_X["city"] == "det"),"city"] = "detroit"
    train_X = train_X[train_X["city"] == "detroit"].copy()
    
    # keep only numeric columns 
    dataTypeSeries = train_X.dtypes
    
    # select only float columns
    train_X  = train_X.select_dtypes(include=['float64'])
    # select columns you want to keep
    train_X.head(2)
    
    # drop late_fee and clean_up_cost - late is telling something about the data outcome we want to predtcit and cleanupcost are all 0
    train_X.drop(["clean_up_cost","late_fee"],axis = 1, inplace = True)
    
    # create the train_y
    train_Y = train_X.copy()
    train_Y.reset_index(inplace=True)

    # drop index for the train data set 
    train_X.reset_index(drop=True, inplace=True)
    
    train_X = train_X.iloc[:,3:].copy()
    
    train_Y = train_Y.iloc[:,0]
    
    # replace the Na values with a high negative number
    train_X.fillna(-10000, inplace=True)

    
    # keep the columns only in the test set - which are also in the train set
    test_X = test
    needed_columns=train_X.columns

    test_X=test_X.loc[:,needed_columns]
    test_X.fillna(-10000, inplace=True)
    # the model
    
    ## find the best parameters
    
    parameters = {'max_depth':range(3,20)}
    clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4)
    clf.fit(X=train_X, y=train_Y)
    tree_model = clf.best_estimator_
    

    # train the model 
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.datasets import make_classification
    clf = RandomForestClassifier(max_depth=6)
    clf.fit(train_X, train_Y)
    
    res = clf.predict_proba(test_X)
    
    tickets = pd.DataFrame(test.loc[:,"ticket_id"])
    results = pd.DataFrame({"results":res[:,1]})
    final = pd.concat([results, tickets ], axis=1)
    final.set_index("ticket_id", inplace=True)
    return pd.Series(final.loc[:,"results"])

In [10]:
bm = blight_model()

In [ ]:
# probability that the ticket will not be paid on time 

In [13]:
print(f"the probability that the ticket will not be paid on time: ticket ID and probability are shown{bm}")
#bm

the probability that the ticket will not be paid on time: ticket ID and probability are shownticket_id
284932    0.100401
285362    0.028086
285361    0.086417
285338    0.101799
285346    0.097905
285345    0.101799
285347    0.083433
285342    0.082866
285530    0.020981
284989    0.037516
285344    0.083433
285343    0.022260
285340    0.036735
285341    0.092179
285349    0.084712
285348    0.078082
284991    0.037516
285532    0.044587
285406    0.035258
285001    0.037516
285006    0.029065
285405    0.029065
285337    0.032432
285496    0.083433
285497    0.101799
285378    0.029065
285589    0.037516
285585    0.078082
285501    0.097905
285581    0.029065
            ...   
376367    0.034082
376366    0.040178
376362    0.040178
376363    0.078082
376365    0.035848
376364    0.036387
376228    0.044331
376265    0.041587
376286    0.340862
376320    0.036387
376314    0.042379
376327    0.254994
376385    0.280476
376435    0.038942
376370    0.275742
376434    0.075727
3764